In [1]:
import os
import requests
import base64

In [32]:
# from config import *

In [2]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
os.environ['OPENAI_API_BASE'] = OPENAI_API_BASE

In [3]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import re

In [4]:
model_name = 'gpt-4-turbo' #'gpt-35-turbo'

In [5]:
# initialize tokenizer
import tiktoken
enc = tiktoken.encoding_for_model(model_name)

In [6]:
llm = AzureChatOpenAI(
    deployment_name=model_name,
    model_name=model_name,
    temperature=0,
)

/home/issa/miniconda3/envs/LangChain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
/home/issa/miniconda3/envs/LangChain/lib/python3.10/site-packages/langchain_community/chat_models/azure_openai.py:167: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://francecentral-openai.openai.azure.com/ to https://francecentral-openai.openai.azure.com/openai.
  warnings.warn(
/home/issa/miniconda3/envs/LangChain/lib/python3.10/site-packages/langc

In [15]:
def transform_func(inputs: dict) -> dict:
    text = inputs["text"]
    
    # replace multiple new lines and multiple spaces with a single one
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"output_text": text}

## Map Reduce

In [7]:
map_prompt = """
Tu es un assistant expert dans le domaine financier et en assurance. Tu sais bien extraire les informations à partir d'un
passage de texte.

Ces informations sont généralement des chiffres, des montants, des dates, et pourraient être autour de:
-Présentation de l'organisme.
-Activité de l'organisme (Chiffre d'affaire, Variations, pourcentages, montants, nombres,
Résultat technique, Résultat financier, Résultat net après impôts).
-Provisions de l'organisation.
-Placements autres que les actifs en represenation de contrats en UC ou indexe.
-Changement de la gouvernance.
-Perspectives vision de l'organisme et projets futurs.

#Règles: 
- Tu ne dois pas modifier les valeurs des données financières (EXP: Chiffres d'affaires, pourcentages, taux, montants, ratio de solvabilité, ratio de couverture du SCR etc)
- N'invente pas des informations qui ni fugrent pas dans le texte.

Ecris moi un résumé  pour le texte suivant:
"{text}"
Résumé:
"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

In [8]:
combine_prompt = """
Tu es un assistant expert dans le domaine financier et en assurance.

#Règles: 
- Tu ne dois pas modifier les valeurs des données financières (EXP: Chiffres d'affaires, pourcentages, taux, montants, ratio de solvabilité, ratio de couverture du SCR etc)

Reproduis moi le texte:
```{text}```
Résumé:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

In [9]:
#This is the summarization chain
summary_chain_map_reduce = load_summarize_chain(llm=llm,
                                     chain_type='map_reduce',
                                     map_prompt=map_prompt_template,
                                     combine_prompt=combine_prompt_template,
#                                      verbose=True
                                    )

In [10]:
def load_and_preprocess(path: str) -> str:
    '''
    Load SFCR report from path
    '''
    loader = PyPDFLoader(path)
    pages = loader.load()
    # Combine the pages, and replace the tabs with spaces
    text = ""

    for page in pages:
        text += page.page_content
    
    text_refined = transform_func({'text': text})
    return text_refined['output_text']

In [11]:
def summarize_map_reduce(summary_chain, text:str) -> str:
    '''
    Use summary_chain to generate a summary for the given SFCR report (text) using Map Reduce
    '''
    #split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10500, chunk_overlap=300)
    docs = text_splitter.create_documents([text])

    #compute number of tokens before the summarisation
    length_before = len(enc.encode(text=text))
    print (f"length of text before summarization: {length_before} tokens")

    #Generate the summary
    output = summary_chain.run(docs)

    #compute number of tokens after the summarisation
    length_after = len(enc.encode(text=output))
    print (f"length of text after summarization: {length_after} tokens")

    return output


In [12]:
def save_text(text, name, version=1):
    '''
    Save generated text
    '''
    with open('generated_files/'+name+'_'+str(version)+'.txt', 'w') as f:
        f.write(text)

def load_text(name, version=1):
    '''
    Load generated text
    '''
    with open('generated_files/'+name+'_'+str(version)+'.txt', 'r') as f:
        return f.read()

In [189]:
text = load_and_preprocess("data/rapport-solvabilite-2021-maif-vie.pdf")
rapport_n_1 = summarize_map_reduce(summary_chain_map_reduce, text)

length of text before summarization: 76613 tokens
length of text after summarization: 803 tokens


In [191]:
# save_text(rapport_n_1, 'maif_vie_2021', version=2)

In [190]:
print(rapport_n_1)

Le rapport sur la solvabilité et la situation financière 2021 de MAIF VIE, approuvé le 30 mars 2022, indique une hausse de 10% des encaissements bruts de réassurance et un résultat net comptable de 50 332 milliers d'euros. La société est dirigée par Dominique Mahé et Hélène N’Diaye et dispose d'une gouvernance solide et d'un système de gestion des risques efficace. MAIF VIE, qui n'est pas spécifiquement exposée aux risques du conflit Russie-Ukraine, est régulée par l'ACPR et auditée par PricewaterhouseCoopers Audit et Mazars. Avec un capital de 122 millions d'euros et plus de 587 400 adhérents, elle propose divers produits d'assurance et d'épargne.

La société affiche des ratios de solvabilité robustes avec 248% pour le SCR et 536% pour le MCR, et des fonds propres de base de niveau 1 de 1,187,963 milliards d'euros. Elle est majoritairement détenue par MAIF et ne distribue pas de dividendes.

En 2021, MAIF VIE a connu une croissance de 5,2% de ses adhérents et une augmentation des frai

In [13]:
rapport_n_1 = load_text('maif_vie_2021', version=2)

In [16]:
text = load_and_preprocess("data/rapport-solvabilite-2022-maif-vie.pdf")
rapport_n = summarize_map_reduce(summary_chain_map_reduce, text)

length of text before summarization: 76920 tokens


/home/issa/miniconda3/envs/LangChain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


length of text after summarization: 2909 tokens


In [20]:
print(rapport_n)

MAIF VIE, basée à Niort depuis 1984 et devenue société à mission en 2021, est une filiale de la MAIF. Sous la supervision de l'ACPR, elle a réalisé en 2022 des encaissements bruts de réassurance de 834 368 milliers d'euros, un taux de rémunération net de 2,10 % et un résultat net comptable de 51 214 milliers d'euros. Yves Pellicier est président du conseil d'administration depuis juin 2022, secondé par les directrices Hélène N’Diaye et Marina Sellari. L'entreprise compte 614 500 adhérents et 222 employés et propose des produits d'épargne, de prévoyance et d'assurance prêt.

Un plan stratégique pour 2023-2026 est en cours, et la gouvernance a été renforcée en 2018 avec l'introduction de quatre fonctions clés de contrôle réglementaires. La masse salariale a connu une hausse de 2,71 % en 2022. Les membres du conseil et les dirigeants respectent des critères de compétence et d'honorabilité et suivent des formations dédiées. La gestion des risques est collaborative et intègre l'ORSA.

Le ra

In [18]:
save_text(rapport_n, 'maif_vie_2022', version=2)

In [19]:
rapport_n = load_text('maif_vie_2022', version=2)

## Rapport de synthèse

In [24]:
# Define prompt
prompt_template = """Tu es un assistant expert dans le domaine financier et en assurance. Tu sais identifier des potentiels
risques dans l'activité d'un organisme. Tu ne peux pas faire des calculs mathématiques complexes sans te servir d'un outil adapté.
Tu as à disposition deux rapport SFCR (solvabilité et sécurité financière) sur l'activité d'une entreprise de deux années
consécutives 'N' et 'N-1'. Ta tâche consiste à générer une fiche de synthèse comparative entre les deux rapports fournis sous 
le format suivant:
'''
1.Présentation [Présente l'organisme, nombre d'assurés, branche d'assurance]
2.Activité [Chiffre d'affaire, Variation par rapport à l'année 'N-1', Résultat technique, financier, net après impôts]
3.Provisionnement [Les provisions de l'ogranisation]
4.Placements [Répartition des placements par classe d'actifs]
5.Gouvernance [Changement de gouvernance]
6.Perspectives [Les projets futurs, la stratégie]
'''

#Règles: 
- Si tu ne trouves pas l'information sur une partie donnée tu peux l'ignorer.
- Si dans le rapport de l'année N il y déjà une comparaison avec l'année N-1 tu dois la garder.

#Rapport de l'année 'N-1'
{rapport_n_1}

#Rapport de l'année 'N'
{rapport_n}

Génération de la fiche synthèse:
"""
# we set the prompt to only have the question we ask
prompt = PromptTemplate(input_variables=['rapport_n_1','rapport_n'], template=prompt_template)

# Define LLM chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [25]:
report = llm_chain.run(rapport_n= rapport_n, rapport_n_1= rapport_n_1)

In [26]:
print(report)

1. Présentation
MAIF VIE est une filiale de la MAIF, spécialisée dans les produits d'épargne, de prévoyance et d'assurance prêt. Basée à Niort depuis 1984, elle est devenue société à mission en 2021. En 2022, elle comptait 614 500 adhérents, en hausse par rapport aux 587 400 adhérents de 2021.

2. Activité
En 2022, MAIF VIE a réalisé des encaissements bruts de réassurance de 834 368 milliers d'euros, avec un taux de rémunération net de 2,10 % et un résultat net comptable de 51 214 milliers d'euros, légèrement supérieur au résultat net de 50 332 milliers d'euros en 2021. La masse salariale a augmenté de 2,71 % en 2022.

3. Provisionnement
Les provisions techniques vie (hors UC et indexés) en 2022 étaient de 9 451 048 euros, avec une meilleure estimation de 9 166 708 euros et une marge de risque de 284 341 euros. Les provisions techniques UC et indexés étaient de 777 106 euros.

4. Placements
En 2022, les placements de MAIF VIE (hors actifs en représentation de contrats en unités de comp

In [27]:
save_text(report, 'report', version=2)

## LLMMath as agent (abandonné)

In [176]:
from langchain.chains import LLMMathChain
llm_math = LLMMathChain(llm=llm, verbose=True)

# llm_math.run("What is 13 raised to the .3432 power?")

/home/issa/miniconda3/envs/LangChain/lib/python3.10/site-packages/langchain/chains/llm_math/base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [177]:
from langchain.agents import Tool

tools = [
    Tool(
        name='outil de Mathématiques',
        func=llm_math.run,
        description=(
            'utilise cet outil lorsque tu veux faire un calcul mathématique'
        )
    )
]

In [178]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)

In [179]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    # early_stopping_method='generate',
    memory=conversational_memory
)

In [180]:
prompt.format_prompt(rapport_n= rapport_n, rapport_n_1= rapport_n_1)

StringPromptValue(text='Tu es un assistant expert dans le domaine financier et en assurance. Tu sais identifier des potentiels\nrisques dans l\'activité d\'un organisme. Tu ne peux pas faire des calculs mathématiques complexes sans te servir d\'un outil adapté.\nTu as à disposition deux rapport SFCR (solvabilité et sécurité financière) sur l\'activité d\'une entreprise de deux années\nconsécutives \'N\' et \'N-1\'. Ta tâche consiste à générer une fiche de synthèse comparative entre les deux rapports fournis sous \nle format suivant:\n\'\'\'\n1.Présentation [Présente l\'organisme, nombre d\'assurés, branche d\'assurance]\n2.Activité [Chiffre d\'affaire, Variation par rapport à l\'année \'N-1\', Résultat technique, financier, net après impôts]\n3.Placements [Répartition des placements par classe d\'actifs]\n4.Gouvernance [Changement de gouvernance]\n5.Perspectives [Les projets futurs, la stratégie]\n\'\'\'\n\n#Règles: \n- Si tu ne trouves pas l\'information sur une partie donnée tu peux 

In [181]:
rapport = agent(prompt.format_prompt(rapport_n= rapport_n, rapport_n_1= rapport_n_1).text)



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "1.Présentation [MAIF VIE, société anonyme d'assurance vie créée en 1984, filiale à 100% de la MAIF, devenue société à mission en 2021, située à Niort. Nombre d'assurés: plus de 614 500 en 2022, croissance de 4,6 %. Branche d'assurance: assurance vie, réassurance, garanties en cas de décès, prévoyance individuelle temporaire, assurances pour prêts.]\n2.Activité [Chiffre d'affaire: 834 368 milliers d'euros en 2022, augmentation par rapport à l'année 'N-1': 2,96% (de 810 412 milliers d'euros en 2021). Résultat net après impôts: 51 214 milliers d'euros en 2022, augmentation par rapport à l'année 'N-1': 1,75% (de 50 332 milliers d'euros en 2021).]\n3.Placements [Répartition des placements par classe d'actifs non spécifiée, mais la société investit selon le principe de la personne prudente avec une exposition modérée aux marchés actions et 89% des actifs financiers dans des supports ISR en 20

In [174]:
print(rapport['output'])

1.Présentation [MAIF VIE, filiale à 100% de la MAIF, créée en 1984, devenue société à mission en 2021, opère sous la directive Solvabilité 2 de l'UE. Nombre d'assurés en N-1: plus de 587 400, en N: plus de 614 500. Branche d'assurance: garanties en cas de décès, prévoyance individuelle temporaire, assurances pour prêts.]


## Affiner les résumés (abandonné)

In [ ]:
combine_prompt = """
Tu es un assistant expert dans le domaine financier et en assurance.

#Règles: 
- Tu ne dois pas modifier les valeurs des données financières (EXP: Chiffres d'affaires, pourcentages, taux, montants, ratio de solvabilité, ratio de couverture du SCR etc)

Ecris moi un résumé du texte suivant avec un titre pour chaque partie du texte:
```{text}```
Résumé:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

In [83]:
# Define prompt
prompt_template_refine = """Tu es un assistant expert dans le domaine financier et en assurance. 
Tu sais bien faire enrichir un résumé à partir du document initial.
Tu as à disposition un résumé et le document initial à partir duquel le résumé a été généré.
Ta tâche consiste à faire enrichir le résumé à disposition avec des informations qui sont manquantes dans le résumé et présentes
dans le document.

Ces informations sont nécessairement des chiffres, des montants, des dates, et pourraient être autour de:
-Présentation de l'organisme.
-Activité de l'organisme (Chiffre d'affaire, Variation par rapport à l'année 'N-1', pourcentages, montants, nombres,
 Résultat technique, Résultat financier, Résultat net après impôts)
-Répartition des placements par classe d'actifs.
-Changement de la gouvernance.
-Perspectives et projets futurs.


#Règles: 
- Tu ne dois pas supprimer les chiffres dans l'ancien résumé ni les modifier.
- Tu dois garder la même structure du résumé donné.

#Document initial
{doc_init}

#Résumé
{resume_}

Génération du nouveau résumé:
"""
# we set the prompt to only have the question we ask
prompt_refine = PromptTemplate(input_variables=['doc_init','resume_'], template=prompt_template_refine)

# Define LLM chain
llm_chain_refine = LLMChain(llm=llm, prompt=prompt_refine)

# res = llm_chain.run(
#     rapport_n_1=rapport_n_1,
#     rapport_n=rapport_n
#     )

In [84]:
def refine_resume(num_iter: int, resume_, doc_init):
    final_resume = resume_
    for _ in range(num_iter):
        final_resume = llm_chain_refine.run(doc_init=doc_init, resume_=final_resume)
    return final_resume

In [85]:
doc_n_1 = load_and_preprocess("data/rapport-solvabilite-2021-maif-vie.pdf")
rapport_n_1_ = refine_resume(1, rapport_n_1, doc_n_1)

In [88]:
print(rapport_n_1_)

MAIF VIE a démontré une bonne performance financière en 2021 avec une croissance de 10% de ses encaissements bruts de réassurance et un bénéfice net de 50 332 milliers d'euros. La société a maintenu une structure de gouvernance solide, avec une séparation claire des rôles de président et de directeur général, et un système de gestion des risques efficace qui lui a permis de ne pas subir d'impact majeur du conflit russo-ukrainien.

La société a respecté les normes Solvabilité 2 avec des ratios de couverture solides, reflétant sa résilience et sa solidité financière. MAIF VIE a également mis en place des politiques et des procédures strictes pour la gestion des fonds propres, assurant ainsi la pérennité de l'entreprise.

Le profil de MAIF VIE est marqué par une large base d'adhérents et une gamme variée de produits d'assurance vie, y compris l'épargne et la prévoyance. La société a également une approche proactive de la gestion des risques, avec des comités spécialisés et des fonctions c